Load the API key and relevant Python libraries

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ.get("OPENAI_API_KEY")

# Set this to `True` if you need GPT4. If not, the code will use GPT-3.5.
GPT4 = False

Helper class

In [ ]:
class Conversation:
    """
    This class helps me keep the context of a conversation. It's not
    sophisticated at all and it simply regulates the number of messages in the
    context window.

    You could try something much more involved, like counting the number of
    tokens and limiting. Even better: you could use the API to summarize the
    context and reduce its length.

    But this is simple enough and works well for what I need.
    """

    messages = None

    def __init__(self):
        # Here is where you can add some personality to your assistant, or
        # play with different prompting techniques to improve your results.
        Conversation.messages = [
            {
                "role": "system",
                "content": (
                    "You are a helpful, polite, old English assistant. Answer "
                    "the user prompt with a bit of humor."
                ),
            }
        ]


    def answer(self, prompt):
        """
        This is the function I use to ask questions.
        """
        self._update("user", prompt)

        response = openai.ChatCompletion.create(
            model="gpt-4" if GPT4 else "gpt-3.5-turbo",
            messages=Conversation.messages,
            temperature=0,
        )

        self._update("assistant", response.choices[0].message.content)

        token_dict = self._get_tokens(response)
        self._save_to_file(response.choices[0].message["content"], token_dict, 'chats.txt')

        return response.choices[0].message["content"], token_dict

    def _update(self, role, content):
        Conversation.messages.append({
            "role": role,
            "content": content,
        })

        # This is a rough way to keep the context size manageable.
        if len(Conversation.messages) > 20:
            Conversation.messages.pop(0)

    
    def _get_tokens(self, response):
        return {
            "prompt_tokens": response["usage"]["prompt_tokens"],
            "completion_tokens": response["usage"]["completion_tokens"],
            "total_tokens": response["usage"]["total_tokens"],
        }
    

    def _save_to_file(self, response, token_dict, filename='chats.txt'):
        with open(filename, 'a') as f:
            for message in Conversation.messages:
                if message['role'] == 'user':
                    f.write('User: ' + message['content'] + '\n')
                elif message['role'] == 'system':
                    f.write('System: ' + message['content'] + '\n')

            f.write('\n')
            f.write('Response: ' '\n' + str(response) + '\n')
            f.write('\n')
            f.write('Token Dict: ' + str(token_dict) + '\n')
            f.write('\n' + '--------------------------------------------' + '\n')
            f.write('\n')

Prompt

In [ ]:
prompt = """
What should I wear to a fancy restaurant?
"""

Create a new instance of `Conversation` whenever you want to clear the context.

In [8]:
conversation = Conversation()

response, token_dict = conversation.answer(prompt)
print(response)
print(token_dict)

I can now ask a question and the API will know what happened before

In [ ]:
conversation.answer("What about an overshirt?")

In [ ]:
conversation.answer("What is 2 + 2")